# Praktikum 3 — DBSCAN dan Deteksi Noise

Pada praktikum ini kita akan mempelajari **DBSCAN** (Density-Based Spatial Clustering of Applications with Noise). Langkah-langkah:
1. Membuat dataset sintetis (`make_blobs`).
2. Standarisasi fitur.
3. Terapkan DBSCAN dan hitung jumlah klaster serta titik *noise*.
4. Evaluasi kualitas klaster menggunakan metrik (karena kita memiliki label asli).
5. Visualisasikan hasil (core samples, non-core samples, noise).


In [ ]:
# Import library utama
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn import metrics
import warnings
warnings.simplefilter('ignore')

## Pembuatan Dataset Sintetis

Kita buat tiga cluster buatan (centers) dan kemudian melakukan standarisasi fitur agar DBSCAN bekerja lebih konsisten terhadap skala.

In [ ]:
# Buat dataset sintetis
centers = [[1, 1], [-1, -1], [1, -1]]
X, labels_true = make_blobs(n_samples=750, centers=centers, cluster_std=0.4, random_state=0)

# Standarisasi fitur (mean=0, std=1)
X = StandardScaler().fit_transform(X)

print('X shape:', X.shape)
print('Unique true labels:', np.unique(labels_true))

## Visualisasi data sintetis
Plot sebaran titik sebelum klasterisasi untuk melihat struktur dasar dataset.

In [ ]:
plt.figure(figsize=(6,5))
plt.scatter(X[:, 0], X[:, 1], s=10)
plt.title('Data Sintetis (standar)')
plt.xlabel('x1')
plt.ylabel('x2')
plt.show()

## Menjalankan DBSCAN
Kita pilih `eps=0.3` dan `min_samples=10` sebagai contoh parameter. Anda bisa bereksperimen dengan nilai lain untuk melihat efeknya.

In [ ]:
# Terapkan DBSCAN
db = DBSCAN(eps=0.3, min_samples=10).fit(X)
labels = db.labels_

# Hitung jumlah klaster (abaikan label -1 yang menandai noise)
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

## Evaluasi Kualitas Klasterisasi
Karena dataset ini sintetis dan kita memiliki `labels_true`, kita dapat menghitung metrik evaluasi klaster seperti homogeneity, completeness, ARI, AMI, dan silhouette score.

In [ ]:
print(f"Homogeneity: {metrics.homogeneity_score(labels_true, labels):.3f}")
print(f"Completeness: {metrics.completeness_score(labels_true, labels):.3f}")
print(f"V-measure: {metrics.v_measure_score(labels_true, labels):.3f}")
print(f"Adjusted Rand Index: {metrics.adjusted_rand_score(labels_true, labels):.3f}")
print(f"Adjusted Mutual Information: {metrics.adjusted_mutual_info_score(labels_true, labels):.3f}")
try:
    s = metrics.silhouette_score(X, labels)
    print(f"Silhouette Coefficient: {s:.3f}")
except Exception as e:
    print('Silhouette Coefficient: could not be computed (maybe only 1 cluster or all points noise)')

## Visualisasi Hasil Klasterisasi DBSCAN
* Core samples ditandai dengan titik yang lebih besar.
* Non-core samples (border) ditandai titik lebih kecil.
* Noise (label `-1`) diberi warna hitam.
Visualisasi ini membantu menginterpretasikan hasil DBSCAN.

In [ ]:
unique_labels = set(labels)
core_samples_mask = np.zeros_like(labels, dtype=bool)
if hasattr(db, 'core_sample_indices_'):
    core_samples_mask[db.core_sample_indices_] = True

plt.figure(figsize=(7,6))
for k in unique_labels:
    if k == -1:
        # black used for noise.
        col = [0, 0, 0, 1]
    else:
        col = plt.cm.Spectral(float(k) / (len(unique_labels) if len(unique_labels)>0 else 1))

    class_member_mask = (labels == k)

    # Plot core samples
    xy = X[class_member_mask & core_samples_mask]
    if xy.size:
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col), markeredgecolor='k', markersize=14)

    # Plot non-core samples
    xy = X[class_member_mask & ~core_samples_mask]
    if xy.size:
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col), markeredgecolor='k', markersize=6)

plt.title(f'Estimated number of clusters: {n_clusters_}')
plt.xlabel('x1')
plt.ylabel('x2')
plt.show()

### Interpretasi singkat
- Titik besar berwarna = **core samples** (inti klaster).
- Titik kecil berwarna = **border/non-core samples**.
- Titik hitam = **noise/outlier**.

Silakan bereksperimen dengan parameter `eps` dan `min_samples` untuk melihat bagaimana jumlah klaster dan noise berubah.